In [ ]:
import numpy as np

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

In [ ]:
from astropy.io import fits

In [ ]:
# import forward test module
import fwtest

In [ ]:
# load `forward' magnitudes module
from forward import mag

In [ ]:
# the SEDs we want to use
sed_fits = fits.open('k_nmf_derived.default.fits')

In [ ]:
# emitted wavelength lambda and SED function f
le = sed_fits[11].data
fe = sed_fits[1].data

In [ ]:
# make sure shapes are ok
np.shape(le), np.shape(fe)

In [ ]:
# number of SEDs
nsed = np.shape(fe)[0]

In [ ]:
# filter bands to use
bands = 'griz'

In [ ]:
# the filters for our instrument
filt_fits = fits.open('STD_BANDPASSES_Y3A2_20170715.fits')

In [ ]:
# get the filter columns
lx = filt_fits[1].data['LAMBDA']
Rx = [filt_fits[1].data[b] for b in bands]

In [ ]:
# redshift range
z = np.linspace(0, 4, 401)

In [ ]:
# test the SED integration for many redshifts
# and a single filter
m = mag.integrate_sed(z, le, fe, lx, Rx)

In [ ]:
# the shape should be (len(z), len(fe), len(Rx))
np.shape(m)

In [ ]:
# plot the integraded SED values
plt.figure(figsize=(8, 6))
for i, b in enumerate(bands):
    plt.subplot(len(bands)//2, np.ceil(len(bands)/(len(bands)//2)), i+1)
    plt.title(r'${}$-band'.format(b))
    plt.xlabel('$z$')
    plt.ylabel('mag')
    for j in range(nsed):
        plt.plot(z, np.log10(m[:,j,i]))
plt.tight_layout()
plt.show()